In [ ]:
import os, numpy, pandas, datetime, time, json, requests, re, configparser
from sklearn.preprocessing import MinMaxScaler
from flask import Flask, jsonify
from flask import request
import json
from keras.models import load_model

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
    return numpy.array(dataX)
        
def load_rnn_model(filepath, dataset, count=1):
    model = load_model(filepath)
    raw_data = dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    predict_data = []
    dataset = scaler.fit_transform(raw_data)
    test_data = create_dataset(dataset, look_back=12)
    test_data = numpy.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))
    testPredict = model.predict(test_data)
    predict_data = scaler.inverse_transform(testPredict)
    return raw_data, predict_data

app = Flask(__name__)
@app.route("/predict", methods=['POST'])
def predict():
    data_list = []
    post_data = request.form.get('data')
    data_list = [[value] for value in post_data.split(',')]
    print(data_list)
    data = numpy.array(data_list)
    raw_data, predict_data = load_rnn_model('/rnn_model_for_deploy.h5', data)
    for datas in predict_data:
        for data in datas:
            predict_data = data
    print('raw_data',raw_data, numpy.shape(raw_data))
    return jsonify({'message': str(predict_data)})

port = os.getenv('PORT', '7500')

if __name__ == "__main__":
    app.run(host = '0.0.0.0', port = int(port))